In [1]:
!pip install torch
!pip install pytorch-lightning --quiet
!pip install transformers --quiet
!pip install torchmetrics
!pip install scikit-multilearn
!pip install pandas
!pip install sklearn
!pip install seaborn
#!pip uninstall sagemaker -y
!pip install sagemaker botocore==1.23.23
#!pip uninstall boto3 botocore -y
#!pip install --no-cache-dir sagemaker==1.72.1 botocore==1.23.23
#!pip install boto3 botocore

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
!nvidia-smi

Wed Dec 22 21:13:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P0    25W /  70W |   2120MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from torchmetrics.functional import accuracy, f1, auroc, precision_recall_curve, average_precision
#from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [3]:
# Read in the cleaned data
data_prefix = 'https://raw.githubusercontent.com/nasa-petal/search-engine/main/data/'
df = pd.read_csv(data_prefix + 'cleaned_leaves.csv')

# Drop all non-feature columns
non_feat = ['y', 'text']
df.drop(non_feat, axis=1, inplace=True)

# Drop all labels with < 20 papers
LABEL_COLUMNS = df.columns.tolist()[:-1]
df.drop([col for col, val in df[LABEL_COLUMNS].sum().iteritems() if val < 25], axis=1, inplace=True)
dropcols = ['protect_from_animals', 'coordinate_by_self-organization', 'maintain_biodiversity', 'compete_within/between_species', 'cooperate_within/between_species']
df.drop(dropcols, axis=1, inplace=True)

#df = df[df.columns[df[LABEL_COLUMNS].sum()>3]]
print(df.shape)
df.head()

(11012, 30)


,distribute_liquids,sense_light_in_the_visible_spectrum,optimize_shape/materials,sense_chemicals,manage_stress/strain,actively_move_through/on_liquids,manage_shear,chemically_assemble_organic_compounds,change_size/shape,attach_temporarily,...,manage_wear,respond_to_signals,protect_from_temperature,physically_assemble_structure,prevent_fracture/rupture,protect_from_microbes,manage_impact,protect_from_excess_liquids,actively_move_through/on_solids,text_raw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,Building a home from foam—túngara frog foam ne...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"A nocturnal mammal, the greater mouse-eared ba..."
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Polarization sensitivity in two species of cut...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Identification and characterization of a multi...
4,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,DIFFERENCES IN POLYSACCHARIDE STRUCTURE BETWEE...


In [4]:
LABEL_COLUMNS = df.columns.tolist()[:-1]

biom = df[df[LABEL_COLUMNS].sum(axis=1) > 0]
nonbiom = df[df[LABEL_COLUMNS].sum(axis=1) == 0]

# remove all non-biomimicry papers from dataset.
df = biom

In [5]:
from skmultilearn.model_selection import iterative_train_test_split

def iterative_train_test_split_dataframe(X, y, test_size):
    df_index = np.expand_dims(X.index.to_numpy(), axis=1)
    df_index_y = np.expand_dims(y.index.to_numpy(), axis=1)
    X_train, y_train, X_test, y_test = iterative_train_test_split(df_index, df_index_y, test_size = test_size)
    X_train = X.loc[X_train[:,0]]
    X_test = X.loc[X_test[:,0]]
    y_train = y.loc[y_train[:,0]]
    y_test = y.loc[y_test[:,0]]
    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = iterative_train_test_split_dataframe(X=df[['text_raw']], y=df[LABEL_COLUMNS], test_size = 0.15)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_test, y_test], axis=1)
print(train_df.shape, val_df.shape)

'''
X_train_val, y_train_val, X_test, y_test = iterative_train_test_split_dataframe(X=df[['text_raw']], y=df[LABEL_COLUMNS], test_size = 0.1)
test_df = pd.concat([X_test, y_test], axis=1)
X_train, y_train, X_val, y_val = iterative_train_test_split_dataframe(X=X_train_val, y=y_train_val, test_size = 0.13)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
#train_df, val_df = train_test_split(df, test_size=0.1)
train_df.shape, val_df.shape, test_df.shape
'''

(643, 30) (114, 30)


"\nX_train_val, y_train_val, X_test, y_test = iterative_train_test_split_dataframe(X=df[['text_raw']], y=df[LABEL_COLUMNS], test_size = 0.1)\ntest_df = pd.concat([X_test, y_test], axis=1)\nX_train, y_train, X_val, y_val = iterative_train_test_split_dataframe(X=X_train_val, y=y_train_val, test_size = 0.13)\ntrain_df = pd.concat([X_train, y_train], axis=1)\nval_df = pd.concat([X_val, y_val], axis=1)\n#train_df, val_df = train_test_split(df, test_size=0.1)\ntrain_df.shape, val_df.shape, test_df.shape\n"

In [6]:
BERT_MODEL_NAME = 'allenai/scibert_scivocab_uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [7]:
MAX_TOKEN_COUNT = 512

In [8]:
class BiomimicryDataset(Dataset):

  def __init__(
    self, 
    data: pd.DataFrame, 
    tokenizer: BertTokenizer, 
    max_token_len: int = 128
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    text = data_row.text_raw
    labels = data_row[LABEL_COLUMNS]

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      text=text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels=torch.FloatTensor(labels)
    )

In [9]:
class BiomimicryDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    self.train_dataset = BiomimicryDataset(
      self.train_df,
      self.tokenizer,
      self.max_token_len
    )

    self.test_dataset = BiomimicryDataset(
      self.test_df,
      self.tokenizer,
      self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

In [10]:
N_EPOCHS = 100
BATCH_SIZE = 12 # smaller is better.

data_module = BiomimicryDataModule(
  train_df,
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [11]:
class BiomimicryTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)    
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):
    
    labels = []
    predictions = []
    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)

    for i, name in enumerate(LABEL_COLUMNS):
      class_roc_auc = auroc(predictions[:, i],  labels[:, i])
      #class_ap = average_precision(predictions[:, i], labels[:, i])
      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)
      #self.logger.experiment.add_scalar(f"{name}_ap/Train", class_ap, self.current_epoch)


  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [12]:
dummy_model = nn.Linear(2, 1)

optimizer = AdamW(params=dummy_model.parameters(), lr=0.001)

warmup_steps = 20
total_training_steps = 100

scheduler = get_linear_schedule_with_warmup(
  optimizer, 
  num_warmup_steps=warmup_steps,
  num_training_steps=total_training_steps
)

learning_rate_history = []

for step in range(total_training_steps):
  optimizer.step()
  scheduler.step()
  learning_rate_history.append(optimizer.param_groups[0]['lr'])

In [13]:
steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [14]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(1060, 5300)

In [15]:
model = BiomimicryTagger(
  n_classes=len(LABEL_COLUMNS),
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps 
)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
!rm -rf lightning_logs/
!rm -rf checkpoints/

In [16]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1, # save top 5 or 10.
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [17]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [18]:
trainer = pl.Trainer(
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=N_EPOCHS,
  gpus=1,
  progress_bar_refresh_rate=30
)

/usr/local/lib/python3.6/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f244dead9b0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f244dead9b0>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.6/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [33]:
trainer.fit(model, data_module)

/usr/local/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 109 M 
1 | classifier | Linear    | 22.3 K
2 | criterion  | BCELoss   | 0     
-----------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
439.763   Total estimated model params size (MB)
/usr/local/lib/python3.6/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /root/lightning_logs/version_0/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 42
/usr/local/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


In [32]:
test = trainer.checkpoint_callback.best_model_path
if not test:
    test = '/root/lightning_logs/version_0/checkpoints/epoch=46-step=2537.ckpt'

trained_model = BiomimicryTagger.load_from_checkpoint(
  test,
  n_classes=len(LABEL_COLUMNS)
)
trained_model.eval()
trained_model.freeze()
# skip to bottom section to generate predictions on unlabelled data.

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

val_dataset = BiomimicryDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

predictions = []
labels = []

for item in val_dataset:
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

In [37]:
THRESHOLD = 0.10
accuracy(predictions, labels, threshold=THRESHOLD)

tensor(0.9486)

In [37]:
print("AUROC per tag")

tag_rocs = []
tag_rocs_dict = {}

for i, name in enumerate(LABEL_COLUMNS):
  tag_auroc = auroc(predictions[:, i], labels[:, i], pos_label=1)
  if not torch.isnan(tag_auroc).item():
    tag_rocs.append(tag_auroc)
    tag_rocs_dict[name] = tag_auroc
  #print(f"{name}: {tag_auroc}")

#tag_rocs_dict = sorted(tag_rocs_dict, key=lambda x: x.item())
tag_rocs_dict = dict(sorted(tag_rocs_dict.items(), key=lambda item: item[1].item(), reverse=True))

for name, tag_auroc in tag_rocs_dict.items():
  print(f"{name}: {tag_auroc}")

mean_auroc = torch.mean(torch.stack(tag_rocs))
print("Mean AUROC: ", mean_auroc)

AUROC per tag
sense_light_in_the_visible_spectrum: 1.0
chemically_assemble_organic_compounds: 1.0
send_light_signals_in_the_visible_spectrum: 1.0
protect_from_chemicals: 0.9963303208351135
actively_move_through/on_liquids: 0.9908257126808167
sense_chemicals: 0.9830247163772583
regulate_reproduction_or_growth: 0.9759680032730103
actively_move_through_gases: 0.9727272987365723
send_chemical_signals: 0.954954981803894
prevent_fracture/rupture: 0.9429011940956116
attach_permanently: 0.910091757774353
respond_to_signals: 0.9045454263687134
attach_temporarily: 0.9002285003662109
protect_from_microbes: 0.8954249024391174
protect_from_temperature: 0.875
distribute_solids: 0.8698412179946899
chemically_break_down_organic_compounds: 0.865909218788147
physically_assemble_structure: 0.8500000238418579
optimize_shape/materials: 0.8409091234207153
manage_stress/strain: 0.8371161222457886
change_size/shape: 0.8304405212402344
distribute_liquids: 0.8120035529136658
manage_impact: 0.8090909719467163
ac

In [38]:
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > .10, upper, lower)

print(classification_report(
  y_true, 
  y_pred, 
  target_names=LABEL_COLUMNS, 
  zero_division=0
))

                                            precision    recall  f1-score   support

                        distribute_liquids       0.54      0.64      0.58        11
       sense_light_in_the_visible_spectrum       1.00      1.00      1.00         1
                  optimize_shape/materials       0.33      0.25      0.29         4
                           sense_chemicals       0.80      0.67      0.73         6
                      manage_stress/strain       0.25      0.43      0.32         7
          actively_move_through/on_liquids       0.71      1.00      0.83         5
                              manage_shear       1.00      0.14      0.25         7
     chemically_assemble_organic_compounds       0.75      1.00      0.86         3
                         change_size/shape       0.71      0.71      0.71         7
                        attach_temporarily       0.62      0.62      0.62        13
                    protect_from_chemicals       0.71      1.00      0.83  

In [39]:
print("AP per tag")

tag_aps = []
tag_aps_dict = {}

for i, name in enumerate(LABEL_COLUMNS):
  tag_ap = average_precision(predictions[:, i], labels[:, i], pos_label=1)
  if not torch.isnan(tag_ap).item():
    tag_aps.append(tag_ap)
    tag_aps_dict[name] = tag_ap
  #print(f"{name}: {tag_ap}")

tag_aps_dict = dict(sorted(tag_aps_dict.items(), key=lambda item: item[1].item(), reverse=True))

for name, tag_ap in tag_aps_dict.items():
  print(f"{name}: {tag_ap}")

map = torch.mean(torch.stack(tag_aps))

def gmean(input_x, dim=0):
    log_x = torch.log(input_x)
    return torch.exp(torch.mean(log_x, dim=dim))

gmap = gmean(torch.stack(tag_aps))

print("MAP: ", map)
print("gMAP: ", gmap)

AP per tag
sense_light_in_the_visible_spectrum: 1.0
chemically_assemble_organic_compounds: 1.0
send_light_signals_in_the_visible_spectrum: 0.9999999403953552
protect_from_chemicals: 0.9428571462631226
sense_chemicals: 0.8333333134651184
protect_from_temperature: 0.8308730125427246
regulate_reproduction_or_growth: 0.8019841313362122
actively_move_through/on_liquids: 0.786190390586853
chemically_break_down_organic_compounds: 0.7658730149269104
prevent_fracture/rupture: 0.7557997703552246
protect_from_microbes: 0.7546887993812561
attach_temporarily: 0.7131129503250122
change_size/shape: 0.6989002823829651
distribute_solids: 0.6736042499542236
physically_assemble_structure: 0.5951288938522339
manage_impact: 0.5784313678741455
actively_move_through_gases: 0.574999988079071
distribute_liquids: 0.5403270125389099
actively_move_through/on_solids: 0.5026556849479675
send_chemical_signals: 0.5
passively_move_through/on_liquids: 0.4903923273086548
manage_stress/strain: 0.3889607787132263
manage_w

In [44]:
# show the top k documents for each topic label. total_relevant are the total # of relevant documents for that label in the ground truth.

val_df_preds = val_df

for i, name in enumerate(LABEL_COLUMNS):
  column_name = 'pred_' + name
  val_df_preds[column_name] = predictions[:,i].numpy()
    
df_tmp = val_df_preds

topk = []
k = 20

for i, label in enumerate(LABEL_COLUMNS):
  col = 'pred_' + label
  df_tmp = df_tmp.sort_values(by=[col], ascending=False)
  i = 1
  total_rel = df_tmp[label].sum()
  for idx, sequence in df_tmp.iterrows():
    title = sequence['text_raw']
    #title_sentence = " ".join(title)
    rel = sequence[label]
    actual_labels = ""
    for label2 in LABEL_COLUMNS:
      if sequence[label] == 1:
        actual_labels = actual_labels + label2 + ' | '
    row = [label, idx, title, sequence[col], i, rel, actual_labels, total_rel]
    topk.append(row)
    i += 1
    if (i > k):
      break;
df_topk_by_topic = pd.DataFrame(topk, columns = ['label','idx','title','score', 'rank', 'truly_relevant?', 'actual_labels', 'total_relevant'])
pd.set_option('display.max_rows', 1320)
pd.set_option("max_colwidth", 150)
df_topk_by_topic

,label,idx,title,score,rank,truly_relevant?,actual_labels,total_relevant
0,distribute_liquids,199,"Epithelial sodium channel (ENaC) family: Phylogeny, structure-function, tissue distribution, and associated inherited diseases.. The epithelial so...",0.883762,1,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
1,distribute_liquids,54,"Lymphatic vascular morphogenesis in development, physiology, and disease. The lymphatic vasculature constitutes a highly specialized part of the v...",0.806618,2,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
2,distribute_liquids,136,Baobab trees (Adansonia) in Madagascar use stored water to flush new leaves but not to support stomatal opening before the rainy season.. Baobab t...,0.791118,3,0,,11
3,distribute_liquids,16,"Function of spiral grain in trees. SummaryThrough spiral grain, conduits for sap lead from each root to all branches. This uniform distribution of...",0.659776,4,0,,11
4,distribute_liquids,159,"Bill sweeping in the spoonbill, Platalea leucordia: evidence for a hydrodynamic function. Abstract Abstract. Spoonbills (Aves: Platalea spp., Cico...",0.582371,5,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
5,distribute_liquids,321,"Behavioral and morphological adaptations of a tropical plant to high rainfall. Machaerium arboireum (Leguminosae), a liana of Panamanian lowland f...",0.565726,6,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
6,distribute_liquids,12,A biomechanical perspective on the role of large stem volume and high water content in baobab trees (Adansonia spp.; Bombacaceae).. The stems of l...,0.488335,7,0,,11
7,distribute_liquids,71,Resilience in social insect infrastructure systems. Both human and insect societies depend on complex and highly coordinated infrastructure system...,0.299212,8,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
8,distribute_liquids,167,Actuation systems in plants as prototypes for bioinspired devices. Plants have evolved a multitude of mechanisms to actuate organ movement. The os...,0.260146,9,0,,11
9,distribute_liquids,231,"On Nonexistence of any λ0-invariand positive harmonic function, a counter example to stroock' conjecture. Consider a sub-Markovian semigroup such ...",0.247542,10,0,,11


In [41]:
df_topk_by_topic[df_topk_by_topic['truly_relevant?'] == 1]

,label,idx,title,score,rank,truly_relevant?,actual_labels,total_relevant
0,distribute_liquids,199,"Epithelial sodium channel (ENaC) family: Phylogeny, structure-function, tissue distribution, and associated inherited diseases.. The epithelial so...",0.883762,1,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
1,distribute_liquids,54,"Lymphatic vascular morphogenesis in development, physiology, and disease. The lymphatic vasculature constitutes a highly specialized part of the v...",0.806618,2,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
4,distribute_liquids,159,"Bill sweeping in the spoonbill, Platalea leucordia: evidence for a hydrodynamic function. Abstract Abstract. Spoonbills (Aves: Platalea spp., Cico...",0.582371,5,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
5,distribute_liquids,321,"Behavioral and morphological adaptations of a tropical plant to high rainfall. Machaerium arboireum (Leguminosae), a liana of Panamanian lowland f...",0.565726,6,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
7,distribute_liquids,71,Resilience in social insect infrastructure systems. Both human and insect societies depend on complex and highly coordinated infrastructure system...,0.299212,8,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
10,distribute_liquids,333,Architecture of the wood-wide web: Rhizopogon spp. genets link multiple Douglas-fir cohorts.. *The role of mycorrhizal networks in forest dynamics...,0.167147,11,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
12,distribute_liquids,233,Inter-plant communication through mycorrhizal networks mediates complex adaptive behaviour in plant communities. Trees can communicate with each o...,0.100714,13,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
15,distribute_liquids,171,Food dissemination in ants: robustness of the trophallactic network against resource quality. ABSTRACT Insect societies are often composed of many...,0.079707,16,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,11
20,sense_light_in_the_visible_spectrum,245,Designing efficient microlens arrays: lessons from Nature. Nature provides a whole host of superior multifunctional structures that can be used as...,0.206351,1,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,1
40,optimize_shape/materials,251,Sympodial construction of Fibonacci-type leaf rosettes in Pinguicula moranensis (Lentibulariaceae).. BACKGROUND AND AIMS\nThe leaf rosettes of the...,0.542112,1,1,distribute_liquids | sense_light_in_the_visible_spectrum | optimize_shape/materials | sense_chemicals | manage_stress/strain | actively_move_throu...,4


In [42]:
print('Number of labels with 1 or more correct papers in the top 20: ', len(df_topk_by_topic[df_topk_by_topic['truly_relevant?'] == 1]['label'].unique()), '/', len(LABEL_COLUMNS))

Number of labels with 1 or more correct papers in the top 20:  29 / 29


In [60]:
#!cp /root/lightning_logs/version_0/checkpoints/epoch=46-step=2537.ckpt /root/scibert-top25-aws-epoch46-122221.ckpt

In [2]:
#import sagemaker
#s3_path_to_data = sagemaker.Session().upload_data(bucket='petal-bucket', 
#                                                  path='/root/scibert-top25-aws-epoch46-122221.ckpt', 
#                                                  key_prefix='multi-label-classifier-models')

In [7]:
#print(s3_path_to_data)

s3://petal-bucket/multi-label-classifier-models/scibert-top25-aws-epoch46-122221.ckpt


In [ ]:
#https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate?subscription-key=&count=5000&expr=And(Ty='0',Or(Composite(J.JN=='biomimetics'), Composite(F.FN=='biology')))&model=latest&count=10&offset=0&attributes=Id,DOI,Ti,VFN,F.FN,AA.AuId,AW,RId

In [33]:
# Read in the cleaned data
data_prefix = 'https://raw.githubusercontent.com/nasa-petal/search-engine/main/data/'
df_ul = pd.read_csv(data_prefix + 'cleaned_unlabelled_biom.csv')

# Drop all non-feature columns
non_feat = ['text']
df_ul.drop(non_feat, axis=1, inplace=True)
df_ul

,text_raw
0,Increased Food and Ecosystem Security via Pere...
1,Molecular Mechanisms of Selenium Tolerance and...
2,Hypoxia and Anoxia Tolerance in the Annual Kil...
3,Cutin and Cutan Biopolymers: Their Role as Nat...
4,The Bacterially Produced Metabolite Violacein ...
...,...
319,Encoding Membrane-Potential-Based Memory withi...
320,Kinship influences social bonds among male sou...
321,Versatility of Turing patterns potentiates rap...
322,Attachment-based mechanisms underlying capture...


In [44]:
ul_df_preds = df_ul

predictions = []

for idx, sequence in df_ul.iterrows():
    title = sequence['text_raw']
    encoding = tokenizer.encode_plus(
      title,
      add_special_tokens=True,
      truncation=True,
      max_length=512,
      return_token_type_ids=False,
      padding="max_length",
      return_attention_mask=True,
      return_tensors='pt',
    )
    
    _, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
    #test_prediction = test_prediction.flatten().numpy()
    predictions.append(test_prediction.flatten())

    
predictions = torch.stack(predictions).detach().cpu()

# show the top k documents for each topic label. total_relevant are the total # of relevant documents for that label in the ground truth.

ul_df_preds = df_ul

for i, name in enumerate(LABEL_COLUMNS):
  column_name = 'pred_' + name
  ul_df_preds[column_name] = predictions[:,i].numpy()

TypeError: list indices must be integers or slices, not tuple

In [47]:
# show the top k documents for each topic label. total_relevant are the total # of relevant documents for that label in the ground truth.
    
df_ul_tmp = ul_df_preds

topk = []
k = 20

for i, label in enumerate(LABEL_COLUMNS):
  col = 'pred_' + label
  df_ul_tmp = df_ul_tmp.sort_values(by=[col], ascending=False)
  i = 1
  for idx, sequence in df_ul_tmp.iterrows():
    title = sequence['text_raw']
    row = [label, idx, title, sequence[col], i]
    topk.append(row)
    i += 1
    if (i > k):
      break;
df_topk_by_topic = pd.DataFrame(topk, columns = ['label','idx','title','score', 'rank'])
pd.set_option('display.max_rows', 1320)
pd.set_option("max_colwidth", 150)
df_topk_by_topic

,label,idx,title,score,rank
0,distribute_liquids,15,"Aerial roots of epiphytic orchids: the velamen radicum and its role in water and nutrient uptake. The velamen radicum, a spongy, usually multiple ...",0.549813,1
1,distribute_liquids,179,Effects of temperature on the cuticular transpiration barrier of two desert plants with water-spender and water-saver strategies. The efficacy of ...,0.483653,2
2,distribute_liquids,164,Self-organization of river vegetation leads to emergent buffering of river flows and water levels. Global climate change is expected to impact hyd...,0.419151,3
3,distribute_liquids,178,Sperm head abnormalities are more frequent in songbirds with more helical sperm: A possible trade‐off in sperm evolution. Sperm morphology varies ...,0.322326,4
4,distribute_liquids,79,Beaver-induced spatiotemporal patch dynamics affect landscape-level environmental heterogeneity. Beavers (Castor sp.) are ecosystem engineers that...,0.233960,5
5,distribute_liquids,277,Encoding memory in tube diameter hierarchy of living flow network. Significance Simple organisms manage to thrive in complex environments. Having ...,0.215211,6
6,distribute_liquids,309,"Self-adaptive and efficient propulsion of Ray sperms at different viscosities enabled by heterogeneous dual helixes. Significance For decades, the...",0.198340,7
7,distribute_liquids,46,Mycorrhizal networks counteract competitive effects of canopy trees on seedling survival.. The dynamics of forest ecosystems depend largely on the...,0.156041,8
8,distribute_liquids,89,Microbial diversity drives carbon use efficiency in a model soil. Empirical evidence for the response of soil carbon cycling to the combined effec...,0.150922,9
9,distribute_liquids,126,Exact Spatiotemporal Dynamics of Confined Lattice Random Walks in Arbitrary Dimensions: A Century after Smoluchowski and Pólya. An exact solution ...,0.147851,10


In [48]:
df_topk_by_topic.to_csv('unlabelled_predictions.csv', index=False)